In [1]:
import numpy as np
from scipy import sparse as ss
from scipy.sparse import csr_matrix as cm
import caffe
from caffe import Net

import sys 
sys.path.append('../src/')
from blob2matrix import *

In [2]:
prototxt='/home/sdu/SPNIF//models/caffemodels/MNIST/lenet.prototxt'
caffe_model='/home/sdu/SPNIF/models/caffemodels/MNIST/lenet.caffemodel' 
net = caffe.Net(prototxt,caffe_model, 0)
[(k,v[0].data.shape) for k,v in net.params.items()]  #查看各层参数规模

output_dir = '/home/sdu/SPNIF/models/convered_models/MNIST/'

In [ ]:
import json

shape_dic = {}
for k,v in net.params.items():
    shape_dic[k] = v[0].data.shape
shape_dic['conv1']
with open('/home/sdu/SPNIF/models/convered_models/MNIST/shape_dic.json','w') as f:
    json.dump(shape_dic,f) 

In [ ]:
def get_threshold(layer_name):
    layer = net.params[layer_name][0].data
    threshold = np.median(abs(layer))
    return threshold
def blob2csr(layer_name,threshold):
    param = net.params[layer_name][0].data
    pm = filter_2_matrix(param)# param matirx
    print pm.shape
    counter = 0
    for i_n in range(pm.shape[0]):
        for i_c in range(pm.shape[1]):
            if abs(pm[i_n][i_c]) <2*threshold:
                pm[i_n][i_c] = 0
                counter = counter +1
    spa_pm = cm(pm)
    ss.save_npz(output_dir + '{}'.format(layer_name),spa_pm,'a')
    zero_persent = (float(counter)/float(pm.size))*100
    print('zero weights percentage in layer {} is: '.format(layer_name) + str(zero_persent)+'%')
    return pm
for k,v in net.params.items():
    threshold = get_threshold(k)
    
    blob2matrix(k)

In [3]:
def blob2matrix(layer_name):
    param = net.params[layer_name][0].data
    pm = filter_2_matrix(param)# param matirx
    
    np.save(output_dir + '{}'.format(layer_name),pm,'a')
    print ('done')
for k,v in net.params.items():
    blob2matrix(k)

In [8]:
conv1 = np.load(output_dir+'conv1.npy')
conv1.shape

(20, 25)

In [ ]:
a = conv1.todense()
a.shape